# Table of Contents
* [Importer les librairies et dataframes](#c1)
* [Exploration de chaque caractéristique extraite](#c2)
    * [Creation des fonctions nécessaires](#s2_0)
    * [Largeur](#s2_1)
    * [Netteté](#s2_2)
    * [Bruit](#s2_3)
    * [Ratio pixels blancs / total](#s2_4)
    * [Ratio pixels noirs / total](#s2_5)
* [Répartitions dans les différents sets](#c3)
* [Corrélation entre les caractéristiques](#c4)
    * [Heatmap des corrélations](#s4_1)
    * [Scatter plots et courbes de densite](#s4_2)
* [LDA](#c5)
* [Comparaison avec IIT](#c5)

## Importer les librairies et dataframes <a class="anchor" id="c1"></a>

On importe ici 5 documents:
- train.txt, test.txt et val.txt, pour récupérer, pour chaque image, sa catégorie et son set
- rvl_cdip_draft1.csv, pour récupérer : format, width, height, mode, nb_pages, nettete, bruit_grain, ratio_b, ratio_n
- rvl_cdip_draft_2.csv, pour récupérer : bloc, motif, num_lines, num_words, digit_ratio, top_margin, bottom_margin, left_margin, right_margin

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import os
import matplotlib.cm as cm
from scipy.stats import gaussian_kde
from matplotlib.lines import Line2D
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import ast
import cv2
from sklearn.preprocessing import RobustScaler
import umap

#################################### A MODIFIER SELON SA PROPRE ARBORESCENCE
project_path = 'D:/Projet/mai25_bds_extraction/' 
####################################
raw_data_path = os.path.join(project_path, 'data', 'raw')
processed_data_path = os.path.join(project_path, 'data', 'processed')
rvl_cdip_images_path = os.path.join(raw_data_path, 'RVL-CDIP', 'images')

In [ ]:
df = pd.read_parquet(os.path.join(project_path, 'data', 'extracted', 'rvl_cdip_all_extracted.parquet'))
df.info()

## Exploration de chaque caractéristique extraite <a class="anchor" id="c2"></a>

### Creation des fonctions nécessaires<a id="s2_0"></a>

In [ ]:
def chercher_extremes(col,seuil,comparaison="sup"):
    """
    Cette fonction permet l'affichage des images (et leur path et nom) pour lesquels un seuil est dépassé pour une colonne de df
    col = nom de la colonne de df considérée
    seuil = valeur du seuil
    comparaison = "sup" si on veut garder les valeurs supérieures au seuil, et "inf" si on veut garder les valeurs inférieures
    """
    if comparaison == "inf":
        paths = df[df[col]<seuil]['relative_path'].to_list()
        file = df[df[col]<seuil]['filename'].to_list()
        cat = df[df[col]<seuil]['cat'].to_list()
    else :
        paths = df[df[col]>seuil]['relative_path'].to_list()
        file = df[df[col]>seuil]['filename'].to_list()
        cat = df[df[col]>seuil]['cat'].to_list()

    # Afficher les images concernées
    for i in range (len(paths)):
        print(paths[i]," : ", file[i],", catégorie : ", cat[i])
        file_path = os.path.join(rvl_cdip_images_path, paths[i], file[i])
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        plt.imshow(image, cmap='gray')
        plt.show()

In [ ]:
def percentile(col,mini,maxi):
    q_min= np.percentile(df[col], mini)
    q_max=np.percentile(df[col], maxi)
    print(maxi-mini,"% des données sont entre ",q_min,' et ', q_max)
    return q_min, q_max

In [ ]:
def histogramme(col, q5, q95):
    # Paramètres pour l'impression
    plt.rcParams.update({
        'font.size': 16,          # Taille générale
        'axes.titlesize': 20,     # Titre du graphique
        'axes.labelsize': 18,     # Titres des axes
        'legend.fontsize': 14,    # Taille de légende
        'xtick.labelsize': 14,
        'ytick.labelsize': 14
    })

    plt.figure(figsize=(12,7))
    plt.hist(df[col], bins=60, color='#15016B')
    plt.axvline(q5, color='#FF5733', linestyle='dashed', linewidth=2, label='5e percentile')
    plt.axvline(q95, color='#FFC300', linestyle='dashed', linewidth=2, label='95e percentile')
    plt.legend()
    plt.xlabel(str(col))
    plt.ylabel("Fréquence")
    plt.title(f"Distribution de la colonne : {col}")
    plt.show()
    


### Largeur <a id="s2_1"></a>

In [ ]:
q5_w, q95_w = percentile('width',5,95)

In [ ]:
# On fait un histogramme en échelle log pour y voir  clair
plt.figure(figsize=(12,7))
plt.rcParams.update({
        'font.size': 16,          # Taille générale
        'axes.titlesize': 20,     # Titre du graphique
        'axes.labelsize': 18,     # Titres des axes
        'legend.fontsize': 14,    # Taille de légende
        'xtick.labelsize': 14,
        'ytick.labelsize': 14
    })
plt.hist(df['width'], bins=300, log=True, color='#15016B')
plt.axvline(q5_w, color='#FF5733', linestyle='dashed', linewidth=2, label='5e percentile')
plt.axvline(q95_w, color='#FFC300', linestyle='dashed', linewidth=2, label='95e percentile')
plt.legend()
plt.xlabel("Largeur")
plt.ylabel("Fréquence (échelle log)")
plt.title("Distribution des largeurs (échelle logarithmique)")
plt.show()

In [ ]:
# images avec une très grande largeur
chercher_extremes('width',2500)

In [ ]:
# images avec la plus petite largeur
chercher_extremes('width',600,comparaison="inf")

### Netteté <a id="s2_2"></a>

In [ ]:
q5_n, q95_n = percentile('sharpness',5,95)
histogramme('sharpness', q5_n, q95_n)

print('###############################')

# images avec la plus basse netteté
chercher_extremes('sharpness',0.00005,comparaison="inf")

print('###############################')

# images avec la plus grande netteté
chercher_extremes('sharpness',0.017,comparaison="sup")

### Bruit <a id="s2_3"></a>

In [ ]:
q5_b, q95_b = percentile('noise',5,95)
histogramme('noise', q5_b, q95_b)

print('###############################')

# images avec le plus faible bruit
chercher_extremes('noise',1.2,comparaison="inf")

print('###############################')

# images avec le plus fort bruit
chercher_extremes('noise',59,comparaison="sup")

In [ ]:
#Voyons les images avec peu de bruit
# Filtrer les lignes où le bruit est inférieur à 0.05 pour aller regarder à quoi correspondent ces images
b_paths = df[df['bruit_grain']<0.05]['relative_path'].to_list()
b_file = df[df['bruit_grain']<0.05]['filename'].to_list()
b_cat = df[df['bruit_grain']<0.05]['cat'].to_list()

# Afficher les chemins relatifs concernés
for i in range (len(b_paths)):
    print(b_paths[i]," : ", b_file[i]," : ", b_cat[i])


### Ratio pixels blancs / total <a id="s2_4"></a>

In [ ]:
q5_rb, q95_rb = percentile('ratio_b',5,95)
histogramme('ratio_b', q5_rb, q95_rb)

print('###############################')

# images avec le plus ratio de blanc
chercher_extremes('ratio_b',0.03,comparaison="inf")

print('###############################')

# images avec le plus fort ratio de blanc
chercher_extremes('ratio_b',0.9995,comparaison="sup")

### Ratio pixels noirs / total <a id="s2_5"></a>

In [ ]:
q5_rn, q95_rn = percentile('ratio_n',5,95)
histogramme('ratio_n', q5_rn, q95_rn)

print('###############################')

# images avec le plus petit ratio de noir
chercher_extremes('ratio_n',0.00003,comparaison="inf")

print('###############################')

# images avec le plus fort ratio de noir
chercher_extremes('ratio_n',0.97,comparaison="sup")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df_extract['ratio_b'], name="ratio pixels blancs", marker_color='rgb(255,195,0)'))
fig.add_trace(go.Histogram(x=df_extract['ratio_n'], name="ratio pixels noirs", marker_color='rgb(21, 1, 107)'))

# Change the bar mode
fig.update_layout(
    #barmode='overlay'
    title=dict(text="Histogramme des ratios du nombre de pixels blancs sur le nombre total de pixels"),
    xaxis=dict(title=dict(text="Ratio"), 
               tickmode='linear',  # Mode linéaire
               dtick=0.05             # Un tick toutes les 1 unité
              ),
    yaxis=dict(title=dict(text="Nombre de documents")),
    legend=dict(title=dict(text="ratios"))
)
fig.update_traces(opacity=0.75)
fig.show()

### Entropie <a id="s2_6"></a>

In [ ]:
df.columns

In [ ]:
q5_e, q95_e = percentile('entropy',5,95)
histogramme('entropy', q5_e, q95_e)

print('###############################')

# images avec le plus petit ratio de noir
chercher_extremes('entropy',0.008,comparaison="inf")

print('###############################')

# images avec le plus fort ratio de noir
chercher_extremes('entropy',5.3,comparaison="sup")

### Nombre de lignes <a id="s2_7"></a>

In [ ]:
q5_nl, q95_nl = percentile('nb_lignes',5,95)
histogramme('nb_lignes', q5_nl, q95_nl)

print('###############################')

# images avec le plus petit ratio de noir
chercher_extremes('nb_lignes',0.008,comparaison="inf")

print('###############################')

# images avec le plus fort ratio de noir
chercher_extremes('nb_lignes',95,comparaison="sup")

### Nombre de colonnes <a id="s2_8"></a>

In [ ]:
q5_nc, q95_nc = percentile('nb_colonnes',5,95)
histogramme('nb_colonnes', q5_nc, q95_nc)

print('###############################')

# images avec le plus petit ratio de noir
#chercher_extremes('nb_colonnes',0,comparaison="inf")

print('###############################')

# images avec le plus fort ratio de noir
chercher_extremes('nb_colonnes',6,comparaison="sup")

### Marges <a id="s2_9"></a>

In [ ]:
df.columns

In [ ]:
q5_tm, q95_tm = percentile('top_marge',5,95)
histogramme('top_marge', q5_tm, q95_tm)

In [ ]:
q5_bm, q95_bm = percentile('bottom_marge',5,95)
histogramme('bottom_marge', q5_bm, q95_bm)

In [ ]:
q5_rm, q95_rm = percentile('right_marge',5,95)
histogramme('right_marge', q5_rm, q95_rm)

In [ ]:
q5_lm, q95_lm = percentile('left_marge',5,95)
histogramme('left_marge', q5_lm, q95_lm)

## Répartitions dans les différents sets <a id="c3"></a>

In [ ]:
def repart(col):
    df['bins']=pd.cut(df[col], 
                        bins=[0, 
                             np.quantile(df[col], 0.1),
                             np.quantile(df[col], 0.2), #en fait, Q1, Q2, Q3, Q4 et Q5 ont la même valeur...
                             np.quantile(df[col], 0.3),
                             np.quantile(df[col], 0.4),
                             np.quantile(df[col], 0.5),
                             np.quantile(df[col], 0.6),
                             np.quantile(df[col], 0.7),
                             np.quantile(df[col], 0.8),
                             np.quantile(df[col], 0.9),
                             df[col].max()], 
                        labels=['min-Q1', 'Q1-Q2', 'Q2-Q3', 'Q3-Q4', 'Q4-Q5', 'Q5-Q6', 'Q6-Q7', 'Q7-Q8', 'Q8-Q9', 'Q9-max'],
                        include_lowest=True)

    # On compte les fichiers par classe et set
    df_counts = df.groupby(['bins', 'set']).size().reset_index(name='count')

    # On calcule la proportion pour chaque classe
    set_order = ['train', 'test', 'val']
    class_order = ['min-Q1', 'Q1-Q2', 'Q2-Q3', 'Q3-Q4', 'Q4-Q5', 'Q5-Q6', 'Q6-Q7', 'Q7-Q8', 'Q8-Q9', 'Q9-max']
    df_counts['set'] = pd.Categorical(df_counts['set'], categories=set_order, ordered=True)
    df_counts['proportion'] = df_counts.groupby('bins')['count'].transform(lambda x: x / x.sum())

    # Graphique en barres empilées, proportions normalisées
    fig = px.bar(
        df_counts,
        x='bins',
        y='proportion',
        color='set',
        category_orders={'bins': class_order, 'set': set_order},
        title="Répartition des fichiers par classe, par set, selon : "+col,
        labels={'proportion': 'Proportion', 'bins': 'Classe'},
        color_discrete_sequence=['rgb(87, 24, 69)', 'rgb(199, 0, 57)', 'rgb(255,87,51)']
    )

    fig.update_layout(barmode='stack', yaxis=dict(tickformat=".0%", title="Proportion"),
                     width=600, height=500)
    fig.show()

### On regarde la répartition des images dans les différents dataframes selon leurs caractéristiques

Pour cela, on va commencer par faire des 'bins' pour nos variables d'intérêt, et pour chaque bin, regarder si son contenu est correctement réparti dans les 3 datasets (en proportion)

In [ ]:
df['width_bins']=pd.cut(df['width'], 
                        bins=[0, 
                             np.quantile(df['width'], 0.1)-1,
                             #np.quantile(df['width'], 0.2), #en fait, Q1, Q2, Q3, Q4 et Q5 ont la même valeur...
                             #np.quantile(df['width'], 0.3),
                             #np.quantile(df['width'], 0.4),
                             np.quantile(df['width'], 0.5),
                             np.quantile(df['width'], 0.6),
                             np.quantile(df['width'], 0.7),
                             #np.quantile(df['width'], 0.8),
                             np.quantile(df['width'], 0.9),
                             df['width'].max()], 
                        labels=['min-Q1', 'Q1-Q5', 'Q5-Q6', 'Q6-Q7', 'Q7-Q9', 'Q9-max'],
                        include_lowest=True)

# On compte les fichiers par classe et set
df_counts = df.groupby(['width_bins', 'set']).size().reset_index(name='count')

# On calcule la proportion pour chaque classe
set_order = ['train', 'test', 'val']
class_order = ['min-Q1', 'Q1-Q5', 'Q5-Q6', 'Q6-Q7', 'Q7-Q9', 'Q9-max']
df_counts['set'] = pd.Categorical(df_counts['set'], categories=set_order, ordered=True)
df_counts['proportion'] = df_counts.groupby('width_bins')['count'].transform(lambda x: x / x.sum())

# Graphique en barres empilées, proportions normalisées
fig = px.bar(
    df_counts,
    x='width_bins',
    y='proportion',
    color='set',
    category_orders={'classe': class_order, 'set': set_order},
    title='Répartition des fichiers par classe de largeur et par set',
    labels={'proportion': 'Proportion', 'width_bins': 'Classe'},
    color_discrete_sequence=['rgb(87, 24, 69)', 'rgb(199, 0, 57)', 'rgb(255,87,51)']
)

fig.update_layout(barmode='stack', yaxis=dict(tickformat=".0%", title="Proportion"),width=600, height=500)
fig.show()

In [ ]:
repart('sharpness')

In [ ]:
repart('noise')

In [ ]:
repart('ratio_b')

In [ ]:
repart('ratio_n')

In [ ]:
repart('entropy')

In [ ]:
repart('nb_lignes')

In [ ]:
df['nb_colonnes_bins']=pd.cut(df['nb_colonnes'], 
                        bins=[0, 1, 2, 3, 4, 5, df['nb_colonnes'].max()], 
                        labels=['0 ou 1', '2', '3', '4', '5', '>5'],
                        include_lowest=True)

# On compte les fichiers par classe et set
df_counts = df.groupby(['nb_colonnes_bins', 'set']).size().reset_index(name='count')

# On calcule la proportion pour chaque classe
set_order = ['train', 'test', 'val']
class_order = ['0', '1', '2', '3', '4', '5', '>5']
df_counts['set'] = pd.Categorical(df_counts['set'], categories=set_order, ordered=True)
df_counts['proportion'] = df_counts.groupby('nb_colonnes_bins')['count'].transform(lambda x: x / x.sum())

# Graphique en barres empilées, proportions normalisées
fig = px.bar(
    df_counts,
    x='nb_colonnes_bins',
    y='proportion',
    color='set',
    category_orders={'ratio_b_bins': class_order, 'set': set_order},
    title='Répartition des fichiers par set selon : nb_colonnes',
    labels={'proportion': 'Proportion', 'ratio_b_bins': 'Classe'},
    color_discrete_sequence=['rgb(87, 24, 69)', 'rgb(199, 0, 57)', 'rgb(255,87,51)']
)

fig.update_layout(barmode='stack', yaxis=dict(tickformat=".0%", title="Proportion"), width=500, height=500)
fig.show()

## Corrélation entre les caractéristiques <a class="anchor" id="c4"></a>

- pour voir si la prédiction des catégories avec nos caractéristiques est possible
- pour voir si nos caractéristiques extraites ne sont pas trop redondantes

### Heatmap des corrélations <a class="anchor" id="c4_1"></a>

In [ ]:
df.columns

In [ ]:
# corrélations entre les caractéristiques extraites:

plt.figure(figsize=(12, 10))
fig = go.Figure(data=go.Heatmap(
                    z=df[['top_marge', 'bottom_marge', 'left_marge', 'right_marge', 'nb_lignes', 'nb_colonnes', 'sharpness', 'noise', 'ratio_b', 'ratio_n', 'entropy']].corr(),
                    x=['top_marge', 'bottom_marge', 'left_marge', 'right_marge', 'nb_lignes', 'nb_colonnes', 'sharpness', 'noise', 'ratio_b', 'ratio_n', 'entropy'],
                    y=['top_marge', 'bottom_marge', 'left_marge', 'right_marge', 'nb_lignes', 'nb_colonnes', 'sharpness', 'noise', 'ratio_b', 'ratio_n', 'entropy'],
                    text=round(df[['top_marge', 'bottom_marge', 'left_marge', 'right_marge', 'nb_lignes', 'nb_colonnes', 'sharpness', 'noise', 'ratio_b', 'ratio_n', 'entropy']].corr(),2),
                    texttemplate="%{text}",
                    colorscale ='RdBu'))

fig.update_layout(title=dict(text="Corrélation des caractéristiques extraites"))
fig.show()



### Scatter plots et courbes de densité <a class="anchor" id="c4_2"></a>
on ne peut pas inclure les catégories dans l'heatmap donc on va déjà regarder ce que ça donne avec des nuages de poinrs

In [ ]:
def densite_90(col1, col2):
    
    # Paramètres pour l'impression
    plt.rcParams.update({
        'font.size': 14,          # Taille générale
        'axes.titlesize': 18,     # Titre du graphique
        'axes.labelsize': 16,     # Titres des axes
        'legend.fontsize': 12,    # Taille de légende
        'xtick.labelsize': 12,
        'ytick.labelsize': 12
    })
    
    # Sous-échantillonnage
    max_points = 5000
    df_reduced = df.groupby('cat').apply(lambda x: x.sample(min(len(x), max_points), random_state=42)).reset_index(drop=True)
    
    # Couleurs
    categories = sorted(df_reduced['cat'].unique())
    cmap = cm.get_cmap('rainbow', len(categories))
    cat_colors = {cat: cmap(i) for i, cat in enumerate(categories)}
    
    # Déterminer dynamiquement la grille pour la densité
    x_min, x_max = df_reduced[col1].min(), df_reduced[col1].max()
    y_min, y_max = df_reduced[col2].min(), df_reduced[col2].max()
    
    # Ajouter un léger padding (5%) pour éviter les bordures trop serrées
    x_pad = (x_max - x_min) * 0.05
    y_pad = (y_max - y_min) * 0.05

    x_grid = np.linspace(x_min - x_pad, x_max + x_pad, 100)
    y_grid = np.linspace(y_min - y_pad, y_max + y_pad, 100)
    X, Y = np.meshgrid(x_grid, y_grid)
    positions = np.vstack([X.ravel(), Y.ravel()])
    
    # Figure adaptée à une page A4 paysage (~11.7 x 8.3 pouces)
    fig = plt.figure(figsize=(11.7, 8.3))  # taille A4 paysage en pouces
    
    # Tracer les contours
    for cat in categories:
        sub_df = df_reduced[df_reduced['cat'] == cat]
        x = sub_df[col1].values
        y = sub_df[col2].values
    
        if len(x) < 10:
            continue
    
        values = np.vstack([x, y])
        kde = gaussian_kde(values)
        Z = np.reshape(kde(positions).T, X.shape)
    
        z_vals = kde(values)
        threshold = np.percentile(z_vals, 10)
    
        plt.contour(
            X, Y, Z,
            levels=[threshold],
            colors=[cat_colors[cat]],
            alpha=0.6,
            linewidths=2.5
        )
    
    # Légende propre
    legend_elements = [Line2D([0], [0], color=cat_colors[cat], lw=3, label=f'Catégorie {cat}') for cat in categories]
    plt.legend(
        handles=legend_elements,
        title="Catégories",
        loc='center left',
        bbox_to_anchor=(1.05, 0.5),
        frameon=True
    )
    # Mise en forme finale
    plt.title("Zones principales de densité par catégorie", pad=20)
    plt.xlabel(col1)
    plt.ylabel(col2)
    plt.xlim(x_min - x_pad, x_max + x_pad)
    plt.ylim(y_min - y_pad, y_max + y_pad)
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.tight_layout()
    
    # Optionnel : sauvegarde en HD pour rapport
    # plt.savefig("densite_categories_A4.png", dpi=300)  # ou .pdf
    
    plt.show()
    
    
        

In [ ]:
from itertools import combinations

# Liste des colonnes à combiner
cols = ['top_marge', 'bottom_marge', 'left_marge', 'right_marge',
        'nb_lignes', 'nb_colonnes', 'sharpness', 'noise',
        'ratio_b', 'ratio_n', 'entropy', 'width']

# Boucle sur toutes les paires uniques (sans ordre)
for col1, col2 in combinations(cols, 2):
    print(f"Graphique : {col1} vs {col2}")
    densite_90(col1, col2)

In [ ]:
densite_90('bottom_marge','nb_lignes')

In [ ]:
densite_90('bottom_marge','entropy')

In [ ]:
densite_90('left_marge','nb_lignes')

In [ ]:
densite_90('nb_lignes','noise')

In [ ]:
densite_90('sharpness','noise')

In [ ]:
densite_90('sharpness','entropy')

In [ ]:
#j'ai voulu le faire avec plotly pour avoir le hover mais du coup ça garde en tête des infos et c'est très très lourd et ça fait freezer le notebook
plt.figure(figsize=(15,10))
plt.scatter(x=df[df['width']<1500]['width'], 
            y=df[df['width']<1500]['bruit_grain'], 
            c=df[df['width']<1500]['cat'],
            s=2
           )
plt.colorbar()
plt.title("Catégories (en couleur) selon la largeur et le bruit")
plt.xlabel("Largeur (<1500)")
plt.ylabel("Bruit/grain")
plt.show()

In [ ]:
# Une palette de couleurs fixes
import matplotlib.cm as cm
categories = sorted(df['cat'].unique())
cmap = cm.get_cmap('rainbow', len(categories))  # nombre de couleurs = nombre de catégories
cat_colors = {cat: cmap(i) for i, cat in enumerate(categories)}

# Boucle sur chaque catégorie unique
for cat in categories:
    sub_df = df[df['cat'] == cat]
    plt.figure(figsize=(8, 6))
    plt.scatter(
        x=sub_df['ratio_n'], 
        y=sub_df['bruit_grain'], 
        color=cat_colors[cat],
        label=f'Catégorie {cat}',
        s=2
    )
    plt.legend(title="Catégories")
    plt.title(f"Catégorie {cat} - largeur vs ratio de pixels noirs")
    plt.xlabel("Ratio de pixels noirs")
    plt.ylabel("Bruit/grain")
    plt.xlim([0, 1])
    plt.ylim([0, 60])
    plt.show()

## LDA <a class="anchor" id="c5"></a>

In [ ]:
df.columns

In [ ]:


# Variables numériques
features = ['top_marge', 'bottom_marge', 'left_marge',
       'right_marge', 'nb_lignes', 'nb_colonnes', 'sharpness', 'noise',
       'ratio_b', 'ratio_n', 'entropy', 'width']
X = df[features].values
y = df['cat'].values

# Appliquer LDA
lda = LinearDiscriminantAnalysis(n_components=2)
X_lda = lda.fit_transform(X, y)

# Génération de couleurs fixes
import matplotlib.cm as cm
categories = sorted(np.unique(y))
cmap = cm.get_cmap('rainbow', len(categories))
cat_colors = {cat: cmap(i) for i, cat in enumerate(categories)}

# Plot 2D
plt.figure(figsize=(12, 8))
for cat in categories:
    idx = (y == cat)
    plt.scatter(X_lda[idx, 0], X_lda[idx, 1], 
                color=cat_colors[cat], 
                s=5, alpha=0.6, 
                label=f'Catégorie {cat}')

plt.xlabel("LD1 (axe le plus discriminant)")
plt.ylabel("LD2 (deuxième axe)")
plt.title("Projection LDA des catégories (5 variables → 2D)")
plt.legend(title="Catégories", markerscale=2, fontsize="small", loc='best')
plt.tight_layout()
plt.show()

In [ ]:
#On refait la même mais avec les courbes de niveau à 90% pour une meilleure lisibilité 

# Variables numériques
features = ['top_marge', 'bottom_marge', 'left_marge',
       'right_marge', 'nb_lignes', 'nb_colonnes', 'sharpness', 'noise',
       'ratio_b', 'ratio_n', 'entropy', 'width']
X = df[features].values
y = df['cat'].values

# Appliquer LDA
lda = LinearDiscriminantAnalysis(n_components=2)
X_lda = lda.fit_transform(X, y)

# Génération de couleurs fixes
import matplotlib.cm as cm
categories = sorted(np.unique(y))
cmap = cm.get_cmap('rainbow', len(categories))
cat_colors = {cat: cmap(i) for i, cat in enumerate(categories)}

# Préparer la figure
plt.figure(figsize=(12, 8))

# Pour chaque catégorie
for cat in categories:
    idx = (y == cat)
    data = X_lda[idx].T  # shape (2, N)
    
    # Estimation de densité
    kde = gaussian_kde(data)
    
    # Créer une grille sur laquelle évaluer la densité
    xmin, xmax = X_lda[:, 0].min(), X_lda[:, 0].max()
    ymin, ymax = X_lda[:, 1].min(), X_lda[:, 1].max()
    xx, yy = np.mgrid[xmin:xmax:200j, ymin:ymax:200j]
    grid = np.vstack([xx.ravel(), yy.ravel()])
    zz = kde(grid).reshape(xx.shape)

    # Déterminer le niveau correspondant à 90% de la densité cumulée
    levels = np.linspace(zz.min(), zz.max(), 100)
    zz_sorted = np.sort(zz.ravel())[::-1]
    cumsum = np.cumsum(zz_sorted)
    cumsum /= cumsum[-1]
    level90 = zz_sorted[np.searchsorted(cumsum, 0.90)]

    # Tracer la courbe de niveau à 90%
    plt.contour(xx, yy, zz, levels=[level90],
                colors=[cat_colors[cat]],
                linewidths=2, alpha=0.5,
                label=f'Catégorie {cat}')

# Légende et titres
plt.xlabel("LD1 (axe le plus discriminant)")
plt.ylabel("LD2 (deuxième axe)")
plt.title("Courbes de niveau (90%) LDA par catégorie")
plt.legend(title="Catégories", fontsize="small", loc='best')
plt.tight_layout()
plt.show()


In [ ]:
#On refait la même mais avec un robust scaler

# Variables numériques
features = ['top_marge', 'bottom_marge', 'left_marge',
       'right_marge', 'nb_lignes', 'nb_colonnes', 'sharpness', 'noise',
       'ratio_b', 'ratio_n', 'entropy', 'width']
X = df[features].values
y = df['cat'].values
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# Appliquer LDA
lda = LinearDiscriminantAnalysis(n_components=2)
X_lda = lda.fit_transform(X_scaled, y)

# Génération de couleurs fixes
import matplotlib.cm as cm
categories = sorted(np.unique(y))
cmap = cm.get_cmap('rainbow', len(categories))
cat_colors = {cat: cmap(i) for i, cat in enumerate(categories)}

# Préparer la figure
plt.figure(figsize=(12, 8))

# Pour chaque catégorie
for cat in categories:
    idx = (y == cat)
    data = X_lda[idx].T  # shape (2, N)
    
    # Estimation de densité
    kde = gaussian_kde(data)
    
    # Créer une grille sur laquelle évaluer la densité
    xmin, xmax = X_lda[:, 0].min(), X_lda[:, 0].max()
    ymin, ymax = X_lda[:, 1].min(), X_lda[:, 1].max()
    xx, yy = np.mgrid[xmin:xmax:200j, ymin:ymax:200j]
    grid = np.vstack([xx.ravel(), yy.ravel()])
    zz = kde(grid).reshape(xx.shape)

    # Déterminer le niveau correspondant à 90% de la densité cumulée
    levels = np.linspace(zz.min(), zz.max(), 100)
    zz_sorted = np.sort(zz.ravel())[::-1]
    cumsum = np.cumsum(zz_sorted)
    cumsum /= cumsum[-1]
    level90 = zz_sorted[np.searchsorted(cumsum, 0.90)]

    # Tracer la courbe de niveau à 90%
    plt.contour(xx, yy, zz, levels=[level90],
                colors=[cat_colors[cat]],
                linewidths=2, alpha=0.5,
                label=f'Catégorie {cat}')

# Légende et titres
plt.xlabel("LD1 (axe le plus discriminant)")
plt.ylabel("LD2 (deuxième axe)")
plt.title("Courbes de niveau (90%) LDA par catégorie")
plt.legend(title="Catégories", fontsize="small", loc='best')
plt.tight_layout()
plt.show()


In [ ]:
# essai avec UMAP

import umap
reducer = umap.UMAP(n_neighbors=30, min_dist=0.1)
X_umap = reducer.fit_transform(X_scaled)

# Réducteur UMAP
reducer = umap.UMAP(n_components=2, n_neighbors=30, min_dist=0.1, random_state=42)

# Transformation
X_umap = reducer.fit_transform(X_scaled)

# Création DataFrame pour faciliter la visualisation
df_umap = pd.DataFrame(X_umap, columns=['UMAP1', 'UMAP2'])
df_umap['cat'] = y  # ajouter les catégories

# Génération de couleurs fixes
import matplotlib.cm as cm
categories = sorted(np.unique(y))
cmap = cm.get_cmap('rainbow', len(categories))
cat_colors = {cat: cmap(i) for i, cat in enumerate(categories)}

# Préparer la figure
plt.figure(figsize=(12, 8))

# Pour chaque catégorie
for cat in categories:
    idx = (y == cat)
    data = X_umap[idx].T  # shape (2, N)
    
    # Estimation de densité
    kde = gaussian_kde(data)
    
    # Créer une grille sur laquelle évaluer la densité
    xmin, xmax = X_umap[:, 0].min(), X_umap[:, 0].max()
    ymin, ymax = X_umap[:, 1].min(), X_umap[:, 1].max()
    xx, yy = np.mgrid[xmin:xmax:200j, ymin:ymax:200j]
    grid = np.vstack([xx.ravel(), yy.ravel()])
    zz = kde(grid).reshape(xx.shape)

    # Déterminer le niveau correspondant à 90% de la densité cumulée
    levels = np.linspace(zz.min(), zz.max(), 100)
    zz_sorted = np.sort(zz.ravel())[::-1]
    cumsum = np.cumsum(zz_sorted)
    cumsum /= cumsum[-1]
    level90 = zz_sorted[np.searchsorted(cumsum, 0.90)]

    # Tracer la courbe de niveau à 90%
    plt.contour(xx, yy, zz, levels=[level90],
                colors=[cat_colors[cat]],
                linewidths=2, alpha=0.5,
                label=f'Catégorie {cat}')

# Légende et titres
plt.xlabel("LD1 (axe le plus discriminant)")
plt.ylabel("LD2 (deuxième axe)")
plt.title("Courbes de niveau (90%) LDA par catégorie")
plt.legend(title="Catégories", fontsize="small", loc='best')
plt.tight_layout()
plt.show()

Wow... Qu'est ce que c'est que ça ??

## Comparaison avec IIT <a class="anchor" id="c5"></a>

In [ ]:
fichier = os.path.join(project_path, 'data', 'extracted', 'df_iit_cdip_basic_features.parquet')
iit_1 = pd.read_parquet(fichier, engine='fastparquet')
iit_1.info()

In [ ]:
iit_1['mode'].value_counts()

In [ ]:
iit_1['mode'].value_counts()

In [ ]:
fichier = os.path.join(project_path, 'data', 'extracted', 'df_iit_cdip_basic_features_INCOMPLET.parquet')
iit = pd.read_parquet(fichier, engine='fastparquet')
iit.info()

In [ ]:
iit[iit['sharpness']>0.08]

In [ ]:
plt.figure(figsize=(12,7))
plt.rcParams.update({
        'font.size': 16,          # Taille générale
        'axes.titlesize': 20,     # Titre du graphique
        'axes.labelsize': 18,     # Titres des axes
        'legend.fontsize': 14,    # Taille de légende
        'xtick.labelsize': 14,
        'ytick.labelsize': 14
    })
plt.hist(df['sharpness'], bins=12, color='#15016B', alpha = 0.5, label='RVL-CDIP', log=True)
plt.hist(iit['sharpness'], bins=60, color='#900C3E', alpha = 0.5 , label='IIT-CDIP', log=True)
plt.legend()
plt.xlabel("Sharpness")
plt.ylabel("Fréquence")
plt.title("Distribution de la netteté")
plt.show()

In [ ]:
plt.figure(figsize=(12,7))
plt.rcParams.update({
        'font.size': 16,          # Taille générale
        'axes.titlesize': 20,     # Titre du graphique
        'axes.labelsize': 18,     # Titres des axes
        'legend.fontsize': 14,    # Taille de légende
        'xtick.labelsize': 14,
        'ytick.labelsize': 14
    })
plt.hist(iit['noise'], bins=60, color='#900C3E', alpha = 0.5 , label='IIT-CDIP', log=True)
plt.hist(df['noise'], bins=60, color='#15016B', alpha = 0.5, label='RVL-CDIP', log=True)

plt.legend()
plt.xlabel("noise")
plt.ylabel("Fréquence")
plt.title("Distribution du bruit")
plt.show()

In [ ]:
plt.figure(figsize=(12,7))
plt.rcParams.update({
        'font.size': 16,          # Taille générale
        'axes.titlesize': 20,     # Titre du graphique
        'axes.labelsize': 18,     # Titres des axes
        'legend.fontsize': 14,    # Taille de légende
        'xtick.labelsize': 14,
        'ytick.labelsize': 14
    })
plt.hist(iit['entropy'], bins=10, color='#900C3E', alpha = 0.5 , label='IIT-CDIP', log=True)
plt.hist(df['entropy'], bins=60, color='#15016B', alpha = 0.5, label='RVL-CDIP', log=True)

plt.legend()
plt.xlabel("entropy")
plt.ylabel("Fréquence")
plt.title("Distribution de l'entropie")
plt.show()